<a href="https://colab.research.google.com/github/jezuzo/Prueba-Teoria/blob/master/negrojack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import files
import os
import zipfile
import shutil
from pathlib import Path
from PIL import Image

In [4]:
files.upload()  # Te pedirá que subas el archivo kaggle.json

# Crear carpeta .kaggle
os.makedirs('/root/.kaggle', exist_ok=True)

# Mover el archivo kaggle.json a la carpeta
!mv kaggle.json /root/.kaggle/

# Establecer permisos adecuados
!chmod 600 /root/.kaggle/kaggle.json

!kaggle datasets download -d gpiosenka/cards-image-datasetclassification

with zipfile.ZipFile("cards-image-datasetclassification.zip","r") as zip_ref:
    zip_ref.extractall("titanic_dataset")

Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/gpiosenka/cards-image-datasetclassification
License(s): CC0-1.0
 89% 341M/385M [00:00<00:00, 1.19GB/s]
100% 385M/385M [00:00<00:00, 1.18GB/s]


In [5]:

# Configuración
input_dir = "titanic_dataset"  # Carpeta original con subcarpetas por clase
output_dir = "cards"   # Carpeta destino con estructura YOLOcd
sets = ["train", "valid"]  # Conjuntos disponibles

# Obtener clases ordenadas
class_names = sorted(os.listdir(os.path.join(input_dir, "train")))
class_to_id = {name: idx for idx, name in enumerate(class_names)}

# Crear carpetas de salida
for subset in sets:
    os.makedirs(f"{output_dir}/images/{subset}", exist_ok=True)
    os.makedirs(f"{output_dir}/labels/{subset}", exist_ok=True)

    for class_name in class_names:
        src_dir = os.path.join(input_dir, subset, class_name)
        if not os.path.exists(src_dir):
            continue

        for file in os.listdir(src_dir):
            if not file.lower().endswith((".jpg", ".jpeg", ".png")):
                continue

            img_path = os.path.join(src_dir, file)
            dest_img_path = os.path.join(output_dir, "images", subset, f"{class_name}_{file}")
            label_path = os.path.join(output_dir, "labels", subset, f"{class_name}_{Path(file).stem}.txt")

            # Copiar imagen
            shutil.copy(img_path, dest_img_path)

            # Leer tamaño de imagen
            with Image.open(img_path) as img:
                w, h = img.size

            # Anotación YOLO (caja que cubre toda la imagen)
            # Formato: class_id center_x center_y width height (en proporción)
            yolo_line = f"{class_to_id[class_name]} 0.5 0.5 1.0 1.0\n"

            with open(label_path, "w") as f:
                f.write(yolo_line)

print("✅ Conversión completada en formato YOLO.")

✅ Conversión completada en formato YOLO.


In [6]:
os.listdir('cards')

['labels', 'images']

In [8]:

from ultralytics import YOLO

model = YOLO("yolov8n.yaml")
files.upload() # Subir cards.yaml
results = model.train(data="cards.yaml",epochs=1)

Saving cards.yaml to cards.yaml
Ultralytics 8.3.141 🚀 Python-3.11.12 torch-2.6.0+cpu CPU (Intel Xeon 2.00GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=cards.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=1, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, p

100%|██████████| 755k/755k [00:00<00:00, 15.1MB/s]

Overriding model.yaml nc=80 with nc=53

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytic

YOLOv8n summary: 129 layers, 3,021,183 parameters, 3,021,167 gradients, 8.3 GFLOPs

Freezing layer 'model.22.dfl.conv.weight'
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 804.0±446.5 MB/s, size: 21.1 KB)


train: Scanning /content/cards/labels/train... 7624 images, 0 backgrounds, 0 corrupt: 100%|██████████| 7624/7624 [00:04<00:00, 1670.97it/s]


...

val: Scanning /content/cards/labels/valid... 265 images, 0 backgrounds, 0 corrupt: 100%|██████████| 265/265 [00:00<00:00, 1439.84it/s]

val: New cache created: /content/cards/labels/valid.cache


Plotting labels to runs/detect/train/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000175, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/detect/train
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/1         0G      3.019      5.081      4.226         42        640:  32%|███▏      | 153/477 [08:21<17:42,  3.28s/it]


KeyboardInterrupt: 